In [1]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import re
import massbalancemachine as mbm
import geopandas as gpd
from scipy.spatial.distance import cdist
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GroupKFold, KFold, train_test_split, GroupShuffleSplit
import csv
import pyproj
from pyproj import Transformer
import salem
import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm
from oggm import cfg, utils, workflow, tasks
import logging
import geopandas as gpd
import xarray as xr

import config
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.geodata import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
# Load one SGI yearly grid:
glacierName = 'morteratsch'
year = 2010
df_y_gl = pd.read_csv(path_glacier_grid_sgi + f'{glacierName}_grid_{year}.csv')

# Add correct RGIs (v6) for each point in the SGI outline:
# Specify the shape filename of the glaciers outline obtained from RGIv6
glacier_outline_fname = '../../../data/GLAMOS/RGI/nsidc0770_11.rgi60.CentralEurope/11_rgi60_CentralEurope.shp'

# Load the target data and the glacier outlines
glacier_outline = gpd.read_file(glacier_outline_fname)

df_y_gl.rename(columns={'RGIId': 'RGIId_old'}, inplace=True)

# Add RGI IDs through intersection with shapefiles:
df_y_gl = mbm.data_processing.utils.get_rgi(data=df_y_gl,
                                            glacier_outlines=glacier_outline)

# Drop points without RGI ID:
df_y_gl = df_y_gl.dropna(subset=['RGIId'])
df_y_gl

,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,POINT_ELEVATION,RGIId_old,POINT_ID,ID,...,tp,slhf,sshf,ssrd,fal,str,u10,v10,pcsr,RGIId
325,2010,9.924482,46.366307,0,2610.698345,-2709.698345,-99.0,RGI60-11.01946,26,25,...,0.005046,-2.135734e+06,-1.255111e+06,1.411811e+07,0.537170,-2.937277e+06,0.093143,-0.364174,204.040733,RGI60-11.01946
326,2010,9.924482,46.366307,0,2610.698345,-2709.698345,-99.0,RGI60-11.01946,26,25,...,0.002070,-8.957792e+05,-4.613325e+05,1.082318e+07,0.660303,-2.949665e+06,0.080532,-0.586275,163.243677,RGI60-11.01946
327,2010,9.924482,46.366307,0,2610.698345,-2709.698345,-99.0,RGI60-11.01946,26,25,...,0.005103,-8.092487e+04,3.230343e+04,6.385886e+06,0.781177,-1.802230e+06,-0.078588,-0.046131,135.569267,RGI60-11.01946
328,2010,9.924482,46.366307,0,2610.698345,-2709.698345,-99.0,RGI60-11.01946,26,25,...,0.004256,-2.185683e+04,5.414123e+04,5.291914e+06,0.821514,-1.632202e+06,-0.311334,-0.668511,109.297750,RGI60-11.01946
329,2010,9.924482,46.366307,0,2610.698345,-2709.698345,-99.0,RGI60-11.01946,26,25,...,0.000980,1.206578e+04,9.781684e+04,7.117506e+06,0.805488,-2.183012e+06,-0.313188,-0.883447,123.699613,RGI60-11.01946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1949826,2010,9.929937,46.426818,0,2610.698345,-583.898345,2026.8,RGI60-11.01946,149987,149986,...,0.006017,-8.008908e+05,-6.007389e+05,1.981328e+07,0.738556,-2.339464e+06,0.001714,-0.233811,274.159839,RGI60-11.01946
1949827,2010,9.929937,46.426818,0,2610.698345,-583.898345,2026.8,RGI60-11.01946,149987,149986,...,0.003942,-1.255881e+06,-6.766634e+05,2.045014e+07,0.632820,-1.907437e+06,0.039917,-0.229606,299.603833,RGI60-11.01946
1949828,2010,9.929937,46.426818,0,2610.698345,-583.898345,2026.8,RGI60-11.01946,149987,149986,...,0.003187,-4.048210e+06,-2.196676e+06,2.209395e+07,0.470103,-3.504886e+06,0.211277,-0.353272,275.146129,RGI60-11.01946
1949829,2010,9.929937,46.426818,0,2610.698345,-583.898345,2026.8,RGI60-11.01946,149987,149986,...,0.006139,-2.963635e+06,-1.684503e+06,1.696570e+07,0.496389,-2.522728e+06,0.146368,-0.161854,217.106935,RGI60-11.01946


In [4]:
# Variables of interest
voi = [
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]

grid_coordinates = df_y_gl[['POINT_LON', 'POINT_LAT',
                            'RGIId']].drop_duplicates()
print(grid_coordinates.RGIId.unique())

# Initialize empty columns in the DataFrame
for var in voi:
    grid_coordinates[var] = np.nan

# Read glacier outline (if needed for spatial context)
glacier_outline = gpd.read_file(glacier_outline_fname)

# Group by RGIId to minimize redundant file operations
grouped = grid_coordinates.groupby("RGIId")

['RGI60-11.01946' 'RGI60-11.02172' 'RGI60-11.02173' 'RGI60-11.02138'
 'RGI60-11.02120' 'RGI60-11.02127']


In [5]:
RGIId = 'RGI60-11.01946'
ds = xr.open_dataset(path_OGGM + f'xr_grids/{RGIId}.nc')
group = grouped.get_group(RGIId)

# Coordinate transformation
transf = pyproj.Transformer.from_proj(
    pyproj.CRS.from_user_input("EPSG:4326"),
    pyproj.CRS.from_user_input(ds.pyproj_srs),
    always_xy=True)

# Transform all points in the group
x_stake, y_stake = transf.transform(group["POINT_LON"].values,
                                    group["POINT_LAT"].values)

stake = ds.sel(x=x_stake, y=y_stake, method="nearest")
# Select variables of interest:
stake_var = stake[voi]
stake_var

<xarray.Dataset> Size: 250GB
Dimensions:                  (y: 144264, x: 144264)
Coordinates:
  * x                        (x) float32 577kB -52.6 -52.6 -52.6 ... 343.4 343.4
  * y                        (y) float32 577kB 5.135e+06 5.135e+06 ... 5.141e+06
Data variables:
    hugonnet_dhdt            (y, x) float32 83GB ...
    consensus_ice_thickness  (y, x) float32 83GB ...
    millan_v                 (y, x) float32 83GB ...
Attributes:
    author:         OGGM
    author_info:    Open Global Glacier Model
    pyproj_srs:     +proj=tmerc +lat_0=0 +lon_0=9.92523 +k=0.9996 +x_0=0 +y_0...
    max_h_dem:      3967.0
    min_h_dem:      2001.0
    max_h_glacier:  3967.0
    min_h_glacier:  2035.0

In [6]:
stake_var.hugonnet_dhdt.plot()

: 

: 